In [170]:
import numpy as np

In [171]:
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import numpy as np

In [184]:
from tensorflow.keras.datasets import boston_housing
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [185]:
class DenseLayer:
    def __init__(self, activation_function, isLastLayer, weightsMatrix, bias):
        self.isLastLayer = isLastLayer
        #self.bias = np.random.randn()*(np.sqrt(2.0/self.units))
        self.activation_function = activation_function
        self.weights = weightsMatrix
        self.bias = bias
        #self.weights = np.random.uniform(self.input_size, self.units)*(np.sqrt(2.0/self.units)) # He initialization (suitable for relu)
    
    def get_output(self, layer_input):
        self.layer_input = layer_input
        
        if self.isLastLayer:
            return np.dot(self.weights, self.layer_input) + self.bias
        else:
            a = np.dot(self.weights, self.layer_input) + self.bias
            return self.activation(a)
    
    def activation(self, xs):
        return np.array(list(map(lambda x: max(x,0.0), xs)))

In [186]:
class NeuralNetwork():
    def __init__(self, code, num_of_layers, num_of_units_per_layer, X, y):
        self.layers = []
        self.loss = None
        self.X = X
        self.y = y
        self.code = code
        self.num_of_layers = num_of_layers
        self.num_of_units_per_layer = num_of_units_per_layer
        self.fitness = self.calcFitness()
        
    def decodeWeightsAddLayers(self):
        for i in range(self.num_of_layers - 1):
            limit = self.num_of_units_per_layer[i] * self.num_of_units_per_layer[i+1] + self.num_of_units_per_layer[i+1]
            weights1 = np.array(self.code[:self.num_of_units_per_layer[i] * self.num_of_units_per_layer[i+1]])
            bias = self.code[self.num_of_units_per_layer[i] * self.num_of_units_per_layer[i+1]:limit]
            
            weightsMatrix = weights1.reshape(self.num_of_units_per_layer[i+1], self.num_of_units_per_layer[i])
            
            isLastLayer = i == self.num_of_layers-2
            
            self.layers.append(DenseLayer('relu', isLastLayer, weightsMatrix, bias))
            
    def __lt__(self, other):
        return self.fitness < other.fitness
            
    def calcFitness(self):
        self.decodeWeightsAddLayers()
        y_pred = self.all_results()
        return self.mse(self.y, y_pred)
        
    def add_layer(self, layer):
        self.layers.append(layer)
        
    def mse(self, y_trues, y_preds):
        return -np.sum((y_trues-y_preds)**2)/len(y_trues)
        
    def mae(self):
        pass
    
    def predict(self, x):
        layer_output = x
        for layer in self.layers:
            layer_output = layer.get_output(layer_output)
        
        return layer_output
    
    def all_results(self):
        results = []
        for i in range(self.X.shape[0]):
            results.append(self.predict(self.X[i])[0])
        return results

In [204]:
num_of_layers = 3 #including input and output
num_of_units_per_layer = [13, 50, 1]

numberOfWeights = 0
for i in range(len(num_of_units_per_layer)-1):
    numberOfWeights += num_of_units_per_layer[i] * num_of_units_per_layer[i+1]
    
numberOfWeights += sum(num_of_units_per_layer) - num_of_units_per_layer[0] # dodao bias za svaki unit
    
initWeights = np.random.uniform(low=-6.0, high=6.0, size=numberOfWeights)
len(initWeights[:num_of_units_per_layer[i] * num_of_units_per_layer[i+1]])
num_of_units_per_layer[0] * num_of_units_per_layer[1]



650

In [205]:
#nn = NeuralNetwork(initWeights, num_of_layers, num_of_units_per_layer, X_train, y_train)

In [213]:
def selection(population):
    TOURNAMENT_SIZE = 20
    bestFitness = float('-inf')
    index = -1
    for i in range(TOURNAMENT_SIZE):
        
        randomIndividual = random.randrange(len(population))
        if population[randomIndividual].fitness > bestFitness:
            bestFitness = population[randomIndividual].fitness
            index = randomIndividual
    return index

In [222]:
def crossover(parent1, parent2, child1, child2):
    breakpoint = random.randrange(len(parent1.code))
    
    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child2.code[:breakpoint] = parent2.code[:breakpoint]
    
    child1.code[breakpoint:] = parent2.code[breakpoint:]
    child2.code[breakpoint:] = parent1.code[breakpoint:]
    
def crossover2(parent1, parent2, child1, child2):
    for i in range(len(parent1.code)):
        r = random.uniform(0.0, 1.0)
        if r < 0.5:
            child1.code[i] = parent1.code[i]
            child2.code[i] = parent2.code[i]
        else:
            child1.code[i] = parent2.code[i]
            child2.code[i] = parent1.code[i]

In [223]:
import random
def mutation(individual):
    MUTATION_PROB = 0.001
    for i in range(len(individual.code)):
        if random.random() < MUTATION_PROB:
            individual.code[i] = random.uniform(-1.0, 1.0)

In [227]:
POPULATION_SIZE = 100
NUM_GENERATIONS = 100
ELITISIM_SIZE = 10

population = [NeuralNetwork(np.random.uniform(low=-1.0, high=1.0, size=numberOfWeights), num_of_layers, num_of_units_per_layer, X_train, y_train) for _ in range(POPULATION_SIZE)]
newPopulation = [NeuralNetwork(np.random.uniform(low=-1.0, high=1.0, size=numberOfWeights), num_of_layers, num_of_units_per_layer, X_train, y_train) for _ in range(POPULATION_SIZE)]

for i in range(NUM_GENERATIONS):
    population.sort(reverse=True)
    newPopulation[:ELITISIM_SIZE] = population[:ELITISIM_SIZE]
    for j in range(ELITISIM_SIZE, POPULATION_SIZE, 2):
        parent1Index = selection(population) # OK
        parent2Index = selection(population) # OK
        
        crossover2(population[parent1Index], population[parent2Index], newPopulation[j], newPopulation[j+1]) # OK

        mutation(newPopulation[j])
        mutation(newPopulation[j+1])

        newPopulation[j] = NeuralNetwork(newPopulation[j].code, num_of_layers, num_of_units_per_layer, X_train, y_train)
        newPopulation[j+1] = NeuralNetwork(newPopulation[j+1].code, num_of_layers, num_of_units_per_layer, X_train, y_train)
        
    population = newPopulation
    print(max(population).fitness)
    
bestIndividual = max(population)
print(f'fitness: {bestIndividual.fitness}')

-119.50651858042384
-95.64532993953576
-78.09679488745655
-51.45742425462074
-38.352756728784136
-31.38900090826903
-29.394711611665105
-27.530821505937958
-25.935372764550014
-23.667756773584927
-21.559431612523763
-20.66114511536902
-20.074480826222633
-19.457646686738066
-18.169573658541914
-17.767929864474382
-16.843848503618073
-16.529153579173958
-16.12912165643657
-15.155709843861398
-14.812240566507427
-14.2609010353996
-13.6006951870346
-13.405712572043125
-13.316409813997687
-13.036478549687926
-12.90678163425471
-12.51194528273878
-12.38016482689692
-12.221692228586441
-12.079427535394657
-11.903383098433565
-11.721784586608539
-11.632637669402618
-11.332391941580056
-11.203345115194265
-11.045435527443038
-10.893936777594234
-10.799040852081506
-10.798826716916588
-10.771990908955708
-10.754518716235946
-10.621288484127415
-10.405988266266661
-10.334836632168965
-10.28018506846079
-10.234652262073022
-10.212642505775802
-10.112625225221894
-10.0609708109665
-10.023671498375

In [221]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
nn1 = NeuralNetwork(bestIndividual.code, num_of_layers, num_of_units_per_layer, X_test, y_test)

In [ ]:
for i in range(102):
    print(y_test[i], nn1.all_results()[i])